## Use `xrefcoord` to Generate Coordinates

When using `Kerchunk` to generate reference datasets for GeoTIFF's, only the dimensions are preserved. `xrefcoord` is a small utility that allows us to generate coordinates for these reference datasets using the geospatial metadata. Similar to other accessor add-on libraries for `Xarray` such as `rioxarray` and `xwrf`, `xrefcord` provides an accessor for an `Xarray` dataset. Importing `xrefcoord` allows us to use the `.xref` accessor to access additional methods. 

In this tutorial we will use the `generate_coords` method to build coordinates for the `Xarray` dataset. `xrefcoord` is *very experimental* and makes assumptions about the underlying data, such as each variable shares the same dimensions etc. Use with caution!


## Overview
   
Within this notebook, we will cover:

1. How to load a Kerchunk reference dataset created from a collection of GeoTIFFs
1. How to use `xrefcoord` to generate coordinates from a GeoTIFF reference dataset

## Prerequisites
| Concepts | Importance | Notes |
| --- | --- | --- |
| [Kerchunk Basics](../foundations/kerchunk_basics) | Required | Core |
| [Xarray Tutorial](https://tutorial.xarray.dev/intro.html) | Required | Core |

- **Time to learn**: 45 minutes
---

In [ ]:
import xarray as xr
import xrefcoord  # noqa

storage_options = {
    "remote_protocol": "s3",
    "skip_instance_cache": True,
}  # options passed to fsspec
open_dataset_options = {"chunks": {}}  # opens passed to xarray

ds = xr.open_dataset(
    "references/RADAR.json",
    engine="kerchunk",
    storage_options=storage_options,
    open_dataset_options=open_dataset_options,
)

In [ ]:
# Generate coordinates from reference dataset
ref_ds = ds.xref.generate_coords(time_dim_name="time", x_dim_name="X", y_dim_name="Y")
# Rename to rain accumulation in 24 hour period
ref_ds = ref_ds.rename({"0": "rr24h"})

## Create a Map

Here we are using `Xarray` to select a single time slice and create a map of 24 hour accumulated rainfall.

In [ ]:
ref_ds["rr24h"].where(ref_ds.rr24h < 60000).isel(time=0).plot(robust=True)

## Create a Time-Series

Next we are plotting accumulated rain as a function of time for a specific point.

In [ ]:
ref_ds["rr24h"][:, 700, 700].plot()